# Teacher fine-tuning

Required Packages

In [1]:
%%time
%%capture

# Install required packages

!pip install transformers
# !pip install datasets
# !pip install fairseq
!pip install sentencepiece

CPU times: user 52.1 ms, sys: 15.4 ms, total: 67.5 ms
Wall time: 21.1 s


## Initialization

In [2]:
# Import required packages

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix
from sklearn.utils import shuffle

from numpy.lib.function_base import average

from tqdm.notebook import tqdm

from collections import Counter

import os
import re
import json
import copy
import collections
import time
import pickle

from transformers import BertConfig, BertTokenizer, BertweetTokenizer, RobertaTokenizer, AlbertTokenizer, DistilBertTokenizer, XLMRobertaTokenizer, XLNetTokenizer, T5Tokenizer
from transformers import BertModel

from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import AutoTokenizer, XLMRobertaTokenizer
from transformers import AutoModelForSequenceClassification, BertForSequenceClassification, DistilBertForSequenceClassification, RobertaForSequenceClassification, AlbertForSequenceClassification, XLMRobertaForSequenceClassification, XLNetForSequenceClassification, T5Model
from transformers import TrainingArguments
from transformers import Trainer
# from fairseq.models.roberta import XLMRModel

In [3]:
tqdm.pandas()

## Train, evaluation, and test sets

### Configurations

In [10]:
TRAIN_PATH = '../input/testinput-1/train.tsv' 
TEST_PATH = '../input/testinput-1/test.tsv'

CONTENT_HEADER = 'sentence'
CONTENT_HEADERS = ['sentence1', 'sentence2']
LABEL_HEADER = 'label'
INDEX_HEADER = 'idx'

# general config
MAX_LEN = 256
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 32
TEST_BATCH_SIZE = 32

EPOCHS = 1
EVERY_EPOCH = 500
LEARNING_RATE = 5e-5

MODELS = ['bert-base-uncased', 'bert-large-uncased', 
          'roberta-base', 'roberta-large', f"cardiffnlp/twitter-roberta-base-sentiment",
          'xlm-roberta-large',
          'xlnet-base-cased', 'xlnet-large-cased',
          't5-base', 't5-large',
          'gpt2-medium']
MODEL_NAMES = ['bert-base-uncased', 'bert-large-uncased', 
          'roberta-base', 'roberta-large', "cardiffnlp-twitter-roberta-base-sentiment",
          'xlm-roberta-large',
          'xlnet-base-cased', 'xlnet-large-cased',
          't5-base', 'tf-large',
          'gpt2-medium']

MODEL_INDEX = 0
OUTPUT_PATH = '/model' + MODEL_NAMES[MODEL_INDEX] + '.bin'
MODEL_PATH =  '/model' + MODEL_NAMES[MODEL_INDEX] + '.pkl'


EVAL_FILE = 'evaluations.csv'
MODEL_RESULTS_FILE = 'model_results.csv'
PREDICTION_FILE = 'model_predictions.tsv'


### Using huggingface datasets

<mark> One of the 'HUGGINGFACE DATASETS' or 'LOCAL DATASETS' should be run

In [39]:
from datasets import load_dataset
train_dataset = load_dataset('glue', 'mrpc', split='train')
eval_dataset = load_dataset('glue', 'mrpc', split='validation')
test_dataset = load_dataset('glue', 'mrpc', split='test')


train = pd.DataFrame(train_dataset)
eval = pd.DataFrame(eval_dataset)
# test = pd.DataFrame(test_dataset).filter([CONTENT_HEADER, INDEX_HEADER])
test = pd.DataFrame(test_dataset).filter([CONTENT_HEADERS[0], CONTENT_HEADERS[1], INDEX_HEADER])

In [40]:
test

,sentence1,sentence2,idx
0,"PCCW 's chief operating officer , Mike Butcher...",Current Chief Operating Officer Mike Butcher a...,0
1,The world 's two largest automakers said their...,Domestic sales at both GM and No. 2 Ford Motor...,1
2,According to the federal Centers for Disease C...,The Centers for Disease Control and Prevention...,2
3,A tropical storm rapidly developed in the Gulf...,A tropical storm rapidly developed in the Gulf...,3
4,The company didn 't detail the costs of the re...,But company officials expect the costs of the ...,4
...,...,...,...
1720,"After Hughes refused to rehire Hernandez , he ...",Hernandez filed an Equal Employment Opportunit...,1720
1721,There are 103 Democrats in the Assembly and 47...,Democrats dominate the Assembly while Republic...,1721
1722,Bethany Hamilton remained in stable condition ...,"Bethany , who remained in stable condition aft...",1722
1723,"Last week the power station ’ s US owners , AE...","The news comes after Drax 's American owner , ...",1723


### Using local files

In [36]:
train = pd.read_csv(TRAIN_PATH,sep='\t')
train[LABEL_HEADER] = train[LABEL_HEADER].progress_apply(lambda l: int(l))


eval = train.sample(n=5000,random_state=42)
train = train.drop(eval.index)
train = train.sample(n=30000,random_state=42)

train.head()

  0%|          | 0/67349 [00:00<?, ?it/s]

,sentence,label
7928,knockaround guys rarely seems interested in ki...,1
17191,"to be compelling , amusing and unsettling at t...",1
19057,"if you go , pack your knitting needles .",0
20911,fascinating stories,1
30223,as if a bored cage spent the duration of the f...,0


In [37]:
eval.head()

,sentence,label
66730,with outtakes in which most of the characters ...,0
29890,enigma is well-made,1
45801,is ) so stoked to make an important film about...,0
29352,the closest thing to the experience of space t...,1
19858,lose their luster,0


In [38]:
test = pd.read_csv(TEST_PATH,sep='\t')
test.head()

,index,sentence
0,0,uneasy mishmash of styles and genres .
1,1,this film 's relationship to actual tension is...
2,2,"by the end of no such thing the audience , lik..."
3,3,director rob marshall went out gunning to make...
4,4,lathan and diggs have considerable personal ch...


### Split data and labels

In [49]:
# x_train, y_train = train[CONTENT_HEADER].values.tolist(), train[LABEL_HEADER].values.tolist()
# x_eval, y_eval = eval[CONTENT_HEADER].values.tolist(), eval[LABEL_HEADER].values.tolist()

# x_test = test[CONTENT_HEADER].values.tolist()
# y_test = [1]* len(x_test)

x_train1, x_train2, y_train = train[CONTENT_HEADERS[0]].values.tolist(), train[CONTENT_HEADERS[1]].values.tolist(), train[LABEL_HEADER].values.tolist()
x_eval1, x_eval2, y_eval = eval[CONTENT_HEADERS[0]].values.tolist(), eval[CONTENT_HEADERS[1]].values.tolist(), eval[LABEL_HEADER].values.tolist()

x_test1, x_test2 = test[CONTENT_HEADERS[0]].values.tolist(), test[CONTENT_HEADERS[1]].values.tolist()
y_test = [1]* len(x_test)

### Showing distributions

In [42]:
train_keys = list(Counter(y_train).keys())
train_values = list(Counter(y_train).values()) # counts the elements' frequency

train_val_dict = {}
for key, value in zip(train_keys, train_values):
    train_val_dict[key] = value

train_distribution = {'class_1': [train_val_dict[1]], 'class_0': [train_val_dict[0]]}
train_distribution = pd.DataFrame(train_distribution)

train_distribution

,class_1,class_0
0,2474,1194


In [43]:
eval_keys = list(Counter(y_eval).keys())
eval_values = list(Counter(y_eval).values()) # counts the elements' frequency

eval_val_dict = {}
for key, value in zip(eval_keys, eval_values):
    eval_val_dict[key] = value

eval_distribution = {'class_1': [eval_val_dict[1]], 'class_0': [eval_val_dict[0]]}
eval_distribution = pd.DataFrame(eval_distribution)

eval_distribution

,class_1,class_0
0,279,129


## Configuration values

In [18]:
model = BertForSequenceClassification.from_pretrained(MODELS[MODEL_INDEX], num_labels=2)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [19]:
tokenizer = AutoTokenizer.from_pretrained(MODELS[MODEL_INDEX])

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

## Creating dataset

In [20]:
class TrainDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

## Tokenization

In [50]:
# train_encodings = tokenizer(x_train, padding=True, truncation=True, max_length=MAX_LEN)

train_encodings = tokenizer(x_train1, x_train2, padding=True, truncation=True, max_length=MAX_LEN)

In [51]:
# train_encodings.keys()
# train_encodings['input_ids'][10]

In [52]:
# eval_encodings = tokenizer(x_eval, padding=True, truncation=True, max_length=MAX_LEN)

eval_encodings = tokenizer(x_eval1, x_eval2, padding=True, truncation=True, max_length=MAX_LEN)

In [53]:
# start_time = time.time()

test_encodings = tokenizer(x_test1, x_test2, padding=True, truncation=True, max_length=MAX_LEN)

# end_time = time.time()
# print(end_time - start_time)

In [54]:
train_dataset = TrainDataset(train_encodings, y_train)

In [55]:
eval_dataset = TrainDataset(eval_encodings, y_eval)

In [56]:
test_dataset = TrainDataset(test_encodings, y_test)

## Fine tuning

In [57]:
# training_args = TrainingArguments("test_trainer")
training_args = TrainingArguments(
      output_dir='output' + MODEL_NAMES[MODEL_INDEX],
      evaluation_strategy="epoch",
      eval_steps=EVERY_EPOCH,
      per_device_train_batch_size=TRAIN_BATCH_SIZE,
      per_device_eval_batch_size=VALID_BATCH_SIZE,
      num_train_epochs=EPOCHS,
      learning_rate=LEARNING_RATE
  )

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [58]:
eval_df = pd.DataFrame()
try:
    eval_df = pd.read_csv(EVAL_FILE)
except:  
    eval_df = pd.DataFrame(y_eval,columns=['real_val'])
    eval_df.to_csv(EVAL_FILE, index=False)

In [59]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='micro')
    precision = precision_score(y_true=labels, y_pred=pred, average='micro')
    f1 = f1_score(labels, pred, average='weighted')

#     eval_df[MODEL_NAMES[MODEL_INDEX]] = pred
#     eval_df.to_csv(EVAL_FILE, index=False)

#     matrix = confusion_matrix(y_true=labels, y_pred=pred)
#     matrix = matrix.diagonal()/matrix.sum(axis=1)
#     matrix = matrix.tolist()

    return {"accuracy": accuracy, "f1_score":f1, "precision": precision} #, "recall": recall, 'matrix': matrix}

In [60]:
trainer = Trainer(
      model=model, 
      args=training_args, 
      train_dataset=train_dataset,
      eval_dataset=eval_dataset,
      compute_metrics=compute_metrics,
  )

In [61]:
training_metrics = trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3668
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 230
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score,Precision
1,No log,0.354887,0.867647,0.863988,0.867647


***** Running Evaluation *****
  Num examples = 408
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)




In [62]:
evaluate_metrics = trainer.evaluate()

***** Running Evaluation *****
  Num examples = 408
  Batch size = 32


In [63]:
training_metrics

TrainOutput(global_step=230, training_loss=0.33080892148225205, metrics={'train_runtime': 42.0822, 'train_samples_per_second': 87.163, 'train_steps_per_second': 5.465, 'total_flos': 194149236639120.0, 'train_loss': 0.33080892148225205, 'epoch': 1.0})

## Evaluation

In [64]:
evaluate_metrics

{'eval_loss': 0.3548870086669922,
 'eval_accuracy': 0.8676470588235294,
 'eval_f1_score': 0.863988207790422,
 'eval_precision': 0.8676470588235294,
 'eval_runtime': 1.2013,
 'eval_samples_per_second': 339.64,
 'eval_steps_per_second': 10.822,
 'epoch': 1.0}

### Saving results in models.csv

In [ ]:
train_loss = training_metrics.metrics['train_loss']
(eval_accuracy, eval_f1, eval_loss) = (evaluate_metrics['eval_accuracy'], evaluate_metrics['eval_f1_score'], evaluate_metrics['eval_loss'])

result_metrics = {'type': [MODEL_NAMES[MODEL_INDEX]],
                'train_loss': ["{:.2f}".format(train_loss)], 
                'eval_loss': ["{:.2f}".format(eval_loss)], 
                'eval_accuracy': ["{:.2f}".format(eval_accuracy*100)], 
                'eval_f1': ["{:.2f}".format(eval_f1*100)],
                'embedding_len': [MAX_LEN], 
                'learning_rate': [LEARNING_RATE], 
                'batch_size': [TRAIN_BATCH_SIZE]}
result_metrics = pd.DataFrame(result_metrics)
result_metrics

In [ ]:
model_df = pd.DataFrame()

try:
    model_df = pd.read_csv(MODEL_RESULTS_FILE)
    model_df = model_df.append(result_metrics)
    model_df.to_csv(MODEL_RESULTS_FILE, index=False)
  # model_df = model_df.iloc[0:0]   #to clear model file
except:
    model_df = model_df.append(result_metrics)
    model_df.to_csv(MODEL_RESULTS_FILE, index=False)

In [ ]:
model_df = pd.read_csv(MODEL_RESULTS_FILE)
model_df.head(50)

## Prediction

In [65]:
test_dataset

In [66]:
predict_metrics = trainer.predict(test_dataset=test_dataset)

***** Running Prediction *****
  Num examples = 1725
  Batch size = 32


In [67]:
predictions = predict_metrics[0]
results = np.argmax(predictions,axis=1)

keys = Counter(results).keys() # equals to list(set(results))
values = Counter(results).values() # counts the elements' frequency
print(keys)
print(values)

dict_keys([1, 0])
dict_values([1193, 532])


In [71]:
test

,sentence1,sentence2,idx,label
0,"PCCW 's chief operating officer , Mike Butcher...",Current Chief Operating Officer Mike Butcher a...,0,1
1,The world 's two largest automakers said their...,Domestic sales at both GM and No. 2 Ford Motor...,1,1
2,According to the federal Centers for Disease C...,The Centers for Disease Control and Prevention...,2,1
3,A tropical storm rapidly developed in the Gulf...,A tropical storm rapidly developed in the Gulf...,3,1
4,The company didn 't detail the costs of the re...,But company officials expect the costs of the ...,4,0
...,...,...,...,...
1720,"After Hughes refused to rehire Hernandez , he ...",Hernandez filed an Equal Employment Opportunit...,1720,0
1721,There are 103 Democrats in the Assembly and 47...,Democrats dominate the Assembly while Republic...,1721,0
1722,Bethany Hamilton remained in stable condition ...,"Bethany , who remained in stable condition aft...",1722,1
1723,"Last week the power station ’ s US owners , AE...","The news comes after Drax 's American owner , ...",1723,1


In [72]:
test[LABEL_HEADER] = results
prediction_df = test.filter(['idx', 'label'])

In [73]:
prediction_df.to_csv(PREDICTION_FILE, index=False)